https://github.com/2noise/ChatTTS/blob/main/README_CN.md

https://github.com/2noise/ChatTTS/issues/216

~~要用chattts环境来跑。~~

In [1]:
import random, os, tqdm, time, json, re, IPython, zhdate, sys
import numpy as np
import pandas as pd
from matplotlib.font_manager import FontProperties
import matplotlib.pyplot as plt
from datetime import datetime

sys.path.append("../../")
# from train_predict_tools_lgbm import *
# from train_predict_tools import * 
# from perf_eval_tools import * 
# from fea_verification import *
# from third_party_data_verify import * 

random.seed(618)
np.random.seed(907)

tqdm.tqdm.pandas() ## 引入这个，就可以在apply的时候用progress_apply了。

sys.path.append("../../../")
new_base_path = os.path.join(
    "/Users/minkexiu/Downloads/GitHub/",
    "/".join(
        os.getcwd().split("/")[-1*(len(sys.path[-1].split("/")) - 1):]
    ),
)
print("storage dir:", new_base_path)
print("code dir:", os.getcwd(), "\n")

## 创建文件夹。
if not os.path.exists(new_base_path):
    os.makedirs(
        new_base_path
    )
if not os.path.exists(os.path.join(new_base_path, "preprocessedData")):
    os.makedirs(
        os.path.join(new_base_path, "preprocessedData")
    )
if not os.path.exists(os.path.join(new_base_path, "originalData")):
    os.makedirs(
        os.path.join(new_base_path, "originalData")
    )
if not os.path.exists(os.path.join(new_base_path, "trained_models")):
    os.makedirs(
        os.path.join(new_base_path, "trained_models")
    )

def load_data_from_newbasepath(filename, dirname = new_base_path, foldername = "originalData", fmt = "csv"):
    valid_format = ["csv", "parquet"]
    assert fmt in valid_format, f"invalid format {fmt}, should be {valid_format}"
    file_path = os.path.join(dirname, foldername, filename + f".{fmt}")
    if fmt == "csv":
        cmd = f'pd.read_{fmt}("{file_path}", quoting=3, lineterminator="\\n")'
    else:
        cmd = f'pd.read_{fmt}("{file_path}")'
    print(cmd)
    return eval(cmd)
def store_data_to_newbasepath(df, filename, dirname = new_base_path, foldername = "preprocessedData", fmt = "parquet", index=False):
    valid_format = ["csv", "parquet"]
    assert fmt in valid_format, f"invalid format {fmt}, should be {valid_format}"
    file_path = os.path.join(dirname, foldername, filename + f".{fmt}")
    cmd = f'df.to_{fmt}("{file_path}", index={index})'
    print(cmd)
    eval(cmd)
    print("data saved.")
    return file_path
def load_data_from_newbasepath__waitUntilDownloaded(filename, dirname = new_base_path, foldername = "originalData", fmt = "csv"):
    flag_path = os.path.join(dirname, foldername, filename + "---downloan_finish_flag.txt")
    print("Downloading, please wait a moment...")
    while True:
#         print(flag_path)
        if os.path.exists(flag_path):
            print("Downloading finished.")
            break
        time.sleep(10)
    valid_format = ["csv", "parquet"]
    assert fmt in valid_format, f"invalid format {fmt}, should be {valid_format}"
    file_path = os.path.join(dirname, foldername, filename + f".{fmt}")
    if fmt == "csv":
        cmd = f'pd.read_{fmt}("{file_path}", quoting=3, lineterminator="\\n")'
    else:
        cmd = f'pd.read_{fmt}("{file_path}")'
    print(cmd)
    return eval(cmd)
def load_data_from_originalData(filename, dirname = new_base_path, foldername = "originalData", fmt = "csv"):
    valid_format = ["csv", "parquet"]
    assert fmt in valid_format, f"invalid format {fmt}, should be {valid_format}"
    file_path = os.path.join(dirname, foldername, filename + f".{fmt}")
    if fmt == "csv":
        cmd = f'pd.read_{fmt}("{file_path}", quoting=3, lineterminator="\\n")'
    else:
        cmd = f'pd.read_{fmt}("{file_path}")'
    print(cmd)
    return eval(cmd)
def load_data_from_preprocessedData(filename, dirname = new_base_path, foldername = "preprocessedData", fmt = "parquet", use_cols = None):
    valid_format = ["csv", "parquet"]
    assert fmt in valid_format, f"invalid format {fmt}, should be {valid_format}"
    file_path = os.path.join(dirname, foldername, filename + f".{fmt}")
    if fmt == "csv":
        if use_cols is None:
            cmd = f'pd.read_{fmt}("{file_path}", quoting=3, lineterminator="\\n")'
        else:
            cmd = f'pd.read_{fmt}("{file_path}", usecols = {use_cols}, quoting=3, lineterminator="\\n")'
    else:
        if use_cols is None:
            cmd = f'pd.read_{fmt}("{file_path}")'
        else:
            cmd = f'pd.read_{fmt}("{file_path}", columns={use_cols})'
    print(cmd)
    return eval(cmd)
def create_originalData_path(filename_or_path):
    return os.path.join(new_base_path, "originalData", filename_or_path)
def create_preprocessedData_path(filename_or_path):
    return os.path.join(new_base_path, "preprocessedData", filename_or_path)
def create_trained_models_path(filename_or_path):
    return os.path.join(new_base_path, "trained_models", filename_or_path)

def run_finish():
    # 假设你的字体文件是 'myfont.ttf' 并且位于当前目录下  
    font = FontProperties(fname="/Users/minkexiu/Documents/GitHub/ML_Tryout/SimHei.ttf", size=24)  
    # 创建一个空白的图形  
    fig, ax = plt.subplots()  
    ax.imshow(
        plt.imread("/Users/minkexiu/Downloads/wallhaven-dgxpyg.jpg")
    )
    # 在图形中添加文字  
    ax.text(
        ax.get_xlim()[1] * 0.5, 
        ax.get_ylim()[0] * 0.5, 
        f"程序于这个点跑完：\n{millisec2datetime(time.time()*1000)}", fontproperties=font, ha="center", va="center", color="red"
    )  
    # 设置图形的布局  
    # ax.set_xlim(0, 1)  
    # ax.set_ylim(0, 1)  
    ax.set_xticks([])  
    ax.set_yticks([])  
    ax.patch.set_color("blue")
    # 显示图形  
    plt.show()

def kill_current_kernel():
    '''杀死当前的kernel释放内存空间。'''
    IPython.Application.instance().kernel.do_shutdown(True) 
    
def simply_show_data(df1):
    print(df1.shape)
    display(df1.head())
    
def wait_flag(saved_flag_path, time_interval_sec=10):
    print("waiting for", saved_flag_path)
    time_count = 0
    while True:
        if os.path.exists(saved_flag_path):
            break
        time.sleep(time_interval_sec)
        time_count+=time_interval_sec
        print(time_count, end=" ")
    print("finish!!")

def parallelly_run_multiple_similar_python_code(codes, nb_workers = 4):
    '''
    codes是多条相似的python代码。
    这个函数的作用就是将其平行地跑，每一条python代码就对应一个线程。或许可以后续优化，比如固定线程数为一个特定值。
    nb_workers 如果赋值为
    '''
    assert (isinstance(nb_workers, int)), "`nb_workers' should be int."
    df_sqls = pd.DataFrame(
        {
            "func": codes

        }
    )
    display(df_sqls)
    from pandarallel import pandarallel
    pandarallel.initialize(nb_workers = df_sqls.shape[0] if nb_workers<0 else nb_workers, progress_bar = True)
    def run_sql_prlly(row):
        try: 
            cmd = f'{row["func"]}'
            print(cmd, "\n")
            eval(cmd)
            return "0-success"
        except Exception as e:
            return e
    df_sqls["run_rsts"] = df_sqls.parallel_apply(lambda row: run_sql_prlly(row), axis = 1)
    display(df_sqls)
    
class TimerContext:  
    def __enter__(self):  
        self.start_time = str(datetime.now())
        print("start time:", self.start_time)
        return self  
    def __exit__(self, exc_type, exc_val, exc_tb):  
        print("start time:", self.start_time)
        print("end time", str(datetime.now()))

def three_num_get_gua(a, b, c):
    '''梅花易数三数起卦，以取本、互、变。'''
    bagua = ["111", "110", "101", "100", "011", "010", "001", "000"]
    guatu = {
        "111": ("☰", "天", "乾金"), 
        "110": ("☱", "泽", "兑金"),
        "101": ("☲", "火", "离火"),
        "100": ("☳" , "雷", "震木"),
        "011": ("☴", "风", "巽木"),
        "010": ("☵", "水", "坎水"),
        "001": ("☶", "山", "艮土"),
        "000": ("☷", "地", "坤土"),
    }
    print(
        "先天八卦数:", ", ".join([f"{i}{guatu[j][-1][0]}"for i, j in zip(range(1,9), bagua)])
    )
    ## https://zhuanlan.zhihu.com/p/457104350
    gua_64 = "天天乾，天风姤，天山遁，天地否，风地观，山地剥，火地晋，火天大有，水水坎，水泽节，水雷屯，水火既济，泽火革，雷火丰，地火明夷，地水师，山山艮，山火贲，山天大畜，山泽损，火泽睽，天泽履，风则中孚，风山渐，雷雷震，雷地豫，雷水解，雷风恒，地风升，水风井，泽风大过，泽雷随，风风巽，风天小畜，风火家人，风雷益，天雷无妄，火雷噬嗑，山雷顾，山风蛊，火火离，火山旅，火风鼎，火水未济，山水蒙，风水涣，天水松，天火同人，地地坤，地雷复，地泽临，地天泰，雷天大壮，泽天夬，水天需，水地比，泽泽兑，泽水困，泽地萃，泽山咸，水山蹇，地山谦，雷山小过，雷泽归妹"
    gua_64_dict = {x[:2]: x[2:]for x in gua_64.split("，")}
    
    shanggua_idx = 7 if (a % 8 == 0) else (a % 8 - 1)
    xiagua_idx = 7 if (b % 8 == 0) else (b % 8 - 1)
    bianyao_idx = 5 if (c % 6 == 0) else (c % 6 - 1)
    print("变爻:", bianyao_idx+1)
    bengua = bagua[xiagua_idx] + bagua[shanggua_idx]
    hugua = bengua[1:-1][:3] + bengua[1:-1][1:]
    biangua = list(bengua)
    biangua[bianyao_idx] = str(1 - int(biangua[bianyao_idx]))
    biangua = "".join(biangua)
    df = pd.DataFrame([[
        guatu[bengua[3:]][0]+guatu[bengua[3:]][2], guatu[hugua[3:]][0]+guatu[hugua[3:]][2], guatu[biangua[3:]][0]+guatu[biangua[3:]][2], 
    ],[
        guatu[bengua[:3]][0]+guatu[bengua[:3]][2], guatu[hugua[:3]][0]+guatu[hugua[:3]][2], guatu[biangua[:3]][0]+guatu[biangua[:3]][2], 
    ]], index=["上卦", "下卦"], columns = [
        guatu[bengua[3:]][1] + guatu[bengua[:3]][1] + gua_64_dict[guatu[bengua[3:]][1] + guatu[bengua[:3]][1]],
        guatu[hugua[3:]][1] + guatu[hugua[:3]][1] + gua_64_dict[guatu[hugua[3:]][1] + guatu[hugua[:3]][1]],
        guatu[biangua[3:]][1] + guatu[biangua[:3]][1] + gua_64_dict[guatu[biangua[3:]][1] + guatu[biangua[:3]][1]],
    ])
    display(df)
    return bengua, hugua, biangua
    
def easy_start_gua():
    """用公历的日、时、分来起卦。"""
    n1, n2, n3 = str(datetime.now())[8:10], str(datetime.now())[11:13], str(datetime.now())[14:16]
    print(n1, n2, n3)
    return three_num_get_gua(int(n1), int(n2), int(n3))
easy_start_gua()

def easy_start_gua_lunar():
    '''用农历的月、日、时辰来起卦。'''
    time_now = datetime.now()
    zh_date_str = str(zhdate.ZhDate.from_datetime(time_now))
    zh_date_str_1 = datetime.strftime(
        datetime(
            *[int(x) for x in re.findall("\d+", zh_date_str)]
        ),
        '%Y-%m-%d'
    )
    zh_hour = (time_now.hour + 1)//2%12+1
    zh_hour_dizhi = "子、丑、寅、卯、辰、巳、午、未、申、酉、戌、亥".split("、")[zh_hour-1]
    
    n1, n2, n3 = zh_date_str_1[5:7], zh_date_str_1[8:10], zh_hour
    print(n1, n2, n3, f"{zh_hour_dizhi}时")
    return three_num_get_gua(int(n1), int(n2), int(n3))
easy_start_gua_lunar()

<>:250: SyntaxWarning: invalid escape sequence '\d'
<>:250: SyntaxWarning: invalid escape sequence '\d'
/var/folders/s1/1jpfx0m52rj4k7cgqkh7g3q40000gn/T/ipykernel_7200/2990379771.py:250: SyntaxWarning: invalid escape sequence '\d'
  *[int(x) for x in re.findall("\d+", zh_date_str)]


storage dir: /Users/minkexiu/Downloads/GitHub/ML_Tryout/LLM/20240604_ChatTTS
code dir: /Users/minkexiu/Documents/GitHub/ML_Tryout/LLM/20240604_ChatTTS 

02 21 21
先天八卦数: 1乾, 2兑, 3离, 4震, 5巽, 6坎, 7艮, 8坤
变爻: 3


,泽风大过,天天乾,泽水困
上卦,☱兑金,☰乾金,☱兑金
下卦,☴巽木,☰乾金,☵坎水


11 02 12 亥时
先天八卦数: 1乾, 2兑, 3离, 4震, 5巽, 6坎, 7艮, 8坤
变爻: 6


,火泽睽,水火既济,雷泽归妹
上卦,☲离火,☵坎水,☳震木
下卦,☱兑金,☲离火,☱兑金


('110101', '101010', '110100')

In [2]:
# ## 如果没有下载过模型，就在这里下载。
# from modelscope import snapshot_download
# model_dir = snapshot_download('pzc163/chatTTS')
# ## 下载的位置在 /Users/minkexiu/.cache/modelscope/hub/pzc163/chatTTS 

# 第一种跑法

In [3]:
import ChatTTS
import torch
import torchaudio
import scipy

In [4]:
import sounddevice as sd
from epub2txt import epub2txt

In [5]:
chat = ChatTTS.Chat()
chat.load(
    compile=True, 
    source="custom", 
    custom_path=create_trained_models_path("chatTTS")
)

True

In [6]:
chunk_size = 50

In [7]:
# from a url to epub
# url = "https://github.com/ffreemt/tmx2epub/raw/master/tests/1.tmx.epub"
# res = epub2txt(url)

# from a local epub file
filepath = r'/Users/minkexiu/Desktop/长城之外-北境与大明边防（1368—1644） (【美】窦德士) (Z-Library).epub'
res = epub2txt(filepath)

# output as a list of chapters
ch_list = epub2txt(filepath, outputlist=True)
# chapter titles will be available as epub2txt.content_titles if available

/opt/anaconda3/envs/ml12/lib/python3.12/site-packages/ebooklib/epub.py:1410: FutureWarning: This search incorrectly ignores the root element, and will be fixed in a future version.  If you rely on the current behaviour, change it to './/xmlns:rootfile[@media-type]'
  for root_file in tree.findall('//xmlns:rootfile[@media-type]', namespaces={'xmlns': NAMESPACES['CONTAINERNS']}):


In [8]:
ori_str = "。".join(ch_list).replace(
    "\n", " "
).replace(
    "\xa0", " "
).replace(
    "1", "一"
).replace(
    "2", "二"
).replace(
    "3", "三"
).replace(
    "4", "四"
).replace(
    "5", "五"
).replace(
    "6", "六"
).replace(
    "7", "七"
).replace(
    "8", "八"
).replace(
    "9", "九"
).replace(
    "0", "零"
).replace(
    ". ", "。"
)

In [9]:
def split_into_chunks_by_sentences(text, chunk_size=150):
    """
    将字符串按照句号拆分，并进一步组合成长度大致为chunk_size的若干子串。

    参数:
    text (str): 要处理的字符串。
    chunk_size (int): 每个子串的大致长度，默认为150。

    返回:
    list: 包含长度大致为chunk_size的子串的列表。
    """
    # 使用句号拆分字符串为句子列表（考虑句号后可能有空格或换行符）
    sentences = [sentence.strip() for sentence in text.split('。') if sentence.strip()]
    
    chunks = []
    current_chunk = []
    current_chunk_length = 0
    
    for sentence in sentences:
        # 计算如果添加当前句子后子串的长度
        new_chunk_length = current_chunk_length + len(sentence) + 1  # +1 是为了考虑添加句号作为分隔符
        
        # 如果添加后长度超过chunk_size，则保存当前子串并开始新的子串
        if new_chunk_length > chunk_size:
            chunks.append('. '.join(current_chunk))
            current_chunk = [sentence]
            current_chunk_length = len(sentence)
        else:
            current_chunk.append(sentence)
            current_chunk_length = new_chunk_length - 1  # 不再需要+1，因为我们在最后会添加句号作为分隔符（但在实际拼接时不加在最后一个句子后）
    
    # 添加最后一个子串（如果有的话）
    if current_chunk:
        chunks.append('. '.join(current_chunk))
    
    return chunks

In [10]:
strs = split_into_chunks_by_sentences(ori_str, 75)

In [12]:
start_end = (10, 20)

In [13]:
to_see = [x.replace(
    "，", "[uv_break]"
).replace(
    "。", "[lbreak]"
).replace(
    "、", "[uv_break]"
).replace(
    " ", "[uv_break]"
) for x in strs[start_end[0]:start_end[1]]]
to_see

['在明朝[uv_break]“鞑靼”多用于指称由北元（逃往漠北的元朝统治集团的后裔）政权统治的蒙古高原东部诸部落[uv_break]他们自称蒙古[uv_break]而明人称之为鞑靼',
 '而下文所见的“瓦剌”多活动在蒙古高原西部[uv_break]元称“斡亦剌惕”[uv_break]明称“瓦剌”[uv_break]清以后称“卫拉特”[uv_break]成为广义的蒙古族的一部分',
 '——译者注[uv_break]从长时段看[uv_break]层出不穷的边防安全事件可分为三个典型时期',
 '第一个典型时期在洪武元年（一三六八年）到宣德十年（一四三五年）[uv_break]这一时期[uv_break]明朝皇帝掌控着全局',
 '太祖[uv_break]永乐皇帝和继任的宣德皇帝以其恩威之势主导着东半球大部的政治格局',
 '太祖和永乐皇帝还曾先后一一次大规模远征漠北[uv_break]这使得鞑靼人（即当时的蒙古人）[uv_break]几乎不怀疑中原政权也具备远程作战的能力',
 '第二个典型时期在正统十四年（一四四九年）后.[uv_break]是年[uv_break]瓦剌领袖也先在土木堡俘虏了年轻的英宗皇帝[uv_break]这一事件成为一大转折点',
 '此后直到隆庆五年（一五七一年）[uv_break]明朝逐渐由攻转守[uv_break]在防线上构筑大量军事防御设施',
 '这一时期[uv_break]皇帝难以恩泽天下[uv_break]明朝逐渐转向内敛并采取闭关锁国政策[uv_break]“华夷之辨”的保守思想抬头[uv_break]渐次取代此前“天下一家”的开放包容氛围',
 '这一趋势在嘉靖皇帝在位时（一五二一—一五六七年）达到峰值[uv_break]中原因而变成一个固若金汤的“大堡垒”']

In [18]:
## https://github.com/2noise/ChatTTS/issues/705

# 默认种子
DEFAULT_AUDIO_SEED_INPUT = 47
DEFAULT_TEXT_SEED_INPUT = 24
DEFAULT_SEED = 425

def generate_audio_file(chat, texts=None, output_path="word_level_output.wav",
                        audio_seed=DEFAULT_AUDIO_SEED_INPUT,
                        text_seed=DEFAULT_TEXT_SEED_INPUT,
                        seed=DEFAULT_SEED):

    torch.manual_seed(seed)

    temperature = 0.3
    top_P = 0.7
    top_K = 20
    refine_text_flag = False

    torch.manual_seed(audio_seed)
    rand_spk = chat.sample_random_speaker()
    params_infer_code = ChatTTS.Chat.InferCodeParams(
        spk_emb=rand_spk,
        temperature=temperature,
        top_P=top_P,
        top_K=top_K)
    # params_refine_text = ChatTTS.Chat.RefineTextParams(
    #     prompt='',) ## [oral_2][laugh_0][break_6]

    torch.manual_seed(text_seed)
    if refine_text_flag:
        text = chat.infer(texts,
                          # skip_refine_text=False,
                          # refine_text_only=True,
                          # params_refine_text=params_refine_text,
                          params_infer_code=params_infer_code
                          )
    else:
        text = texts

    wavs = chat.infer(text,
                      # skip_refine_text=True,
                      # params_refine_text=params_refine_text,
                      params_infer_code=params_infer_code,)
    
    return wavs

In [19]:
wavs = generate_audio_file(
    chat, 
    to_see, 
)

found invalid characters: {'）', '”', '（', '“'}
found invalid characters: {'”', '“'}
found invalid characters: {'—'}
found invalid characters: {'）', '（'}
found invalid characters: {'）', '（'}
found invalid characters: {'）', '（'}
found invalid characters: {'）', '（'}
found invalid characters: {'”', '“'}
found invalid characters: {'—', '“', '）', '”', '（'}
text:  19%|███████                              | 73/384(max) [00:07,  9.59it/s]
code:  28%|█████████▉                         | 582/2048(max) [01:41,  5.73it/s]


In [20]:
scipy.io.wavfile.write(
    filename=create_preprocessedData_path(f"x.wav"), 
    rate=24_000, 
    data=np.concatenate([np.concatenate([wavs[i].T, np.zeros(10000)]) for i in range(len(to_see))]) 
)

In [ ]:
import wave
import numpy as np
 
def shorten_flat_regions(input_file, threshold, output_file):
    # 打开波形文件
    with wave.open(input_file, 'rb') as wav_read:
        params = wav_read.getparams()
        nchannels, sampwidth, framerate, nframes = params[:4]
        
        # 读取波形数据
        str_data = wav_read.readframes(nframes)
        wave_data = np.frombuffer(str_data, dtype=np.int16)
        
        # 将数据转换为一维数组
        wave_data = wave_data * (1.0 / max(abs(wave_data)))
        
        # 找到连续的零或者在阈值以下的点
        crossings = np.where(np.diff(np.abs(wave_data) < threshold))[0] + 1
        
        # 计算每个平波区域的长度
        flat_regions = np.diff(crossings)
        
        # 缩短平波区域，这里以去掉25%的长度为例
        new_flat_regions = [int(len * 0.75) for len in flat_regions]
        
        # 构造新的波形数据
        new_wave_data = []
        start = 0
        for length in new_flat_regions:
            new_wave_data.extend(wave_data[start:start + length])
            start += length
        
        # 将新数据写入新的波形文件
        with wave.open(output_file, 'wb') as wav_write:
            wav_write.setnchannels(nchannels)
            wav_write.setsampwidth(sampwidth)
            wav_write.setframerate(framerate)
            wav_write.writeframes(np.array(new_wave_data).tobytes())
 
# 使用函数处理波形文件
# 假设阈值设为0.1，输入输出文件分别为'input.wav'和'output.wav'
shorten_flat_regions(create_preprocessedData_path(f"x.wav"), 0.1, 'output.wav')

# 下面的代码的接口过时了

In [4]:
## 如果pip装chattts库的话，可以用modelscope先把模型下载下来，然后再跑下面代码：
import ChatTTS
import scipy
model_path = create_trained_models_path("pzc163/chatTTS") # '/Users/minkexiu/.cache/modelscope/hub/pzc163/chatTTS' ## 下载下来的模型的路径。
chat = ChatTTS.Chat()
chat.load(
    vocos_config_path=f"{model_path}/config/vocos.yaml",
    vocos_ckpt_path=f"{model_path}/asset/Vocos.pt",
    dvae_config_path=f"{model_path}/config/dvae.yaml",
    dvae_ckpt_path=f"{model_path}/asset/DVAE.pt",
    gpt_config_path=f"{model_path}/config/gpt.yaml",
    gpt_ckpt_path=f"{model_path}/asset/GPT.pt",
    decoder_config_path=f"{model_path}/config/decoder.yaml", 
    decoder_ckpt_path=f"{model_path}/asset/Decoder.pt", 
    tokenizer_path=f"{model_path}/asset/tokenizer.pt", 
    device='cpu'
)

TypeError: Chat.load() got an unexpected keyword argument 'vocos_config_path'

In [5]:
texts = ["卧槽这个玩意儿有点东西啊[uv_break]，不过暂时也开发不出更多玩法了，仅此而已罢了。能给我来点说唱吗[laugh]", ]
wavs = chat.infer(texts, use_decoder=True)
scipy.io.wavfile.write(filename="./tts3.wav", rate=24_000, data=wavs[0].T)

INFO:ChatTTS.core:All initialized.
 23%|█████████▎                              | 479/2048 [00:17<00:58, 26.82it/s]


In [6]:
texts = ["呦呦切克闹，爱你的猫抛瓦", ]
wavs = chat.infer(texts, use_decoder=True)
scipy.io.wavfile.write(filename="./inmp.wav", rate=24_000, data=wavs[0].T)

INFO:ChatTTS.core:All initialized.
  9%|███▊                                    | 192/2048 [00:06<01:02, 29.55it/s]


In [7]:
texts = ["[laugh]", ]
wavs = chat.infer(texts, use_decoder=True)
scipy.io.wavfile.write(filename="./laugh.wav", rate=24_000, data=wavs[0].T)

INFO:ChatTTS.core:All initialized.
  9%|███▋                                    | 191/2048 [00:06<01:05, 28.43it/s]


# 第二种跑法

In [5]:
## 把ChatTTS库下载到本地解压，然后在ChatTTS文件夹外设置一个代码，删掉pip装的ChatTTS库，
## 跑下一个代码，可能也行。

# Import necessary libraries and configure settings
import torch
import torchaudio
torch._dynamo.config.cache_size_limit = 64
torch._dynamo.config.suppress_errors = True
torch.set_float32_matmul_precision('high')

import ChatTTS
from IPython.display import Audio

# Initialize and load the model: 
chat = ChatTTS.Chat()
chat.load_models(
    # source='local', 
    # local_path="/Users/minkexiu/.cache/modelscope/hub/pzc163/chatTTS"
) # Set to True for better performance

# Define the text input for inference (Support Batching)
texts = [
"So we found being competitive and collaborative was a huge way of staying motivated towards our goals, so one person to call when you fall off, one person who gets you back on then one person to actually do the activity with.",
]

# Perform inference and play the generated audio
wavs = chat.infer(texts)
Audio(wavs[0], rate=24_000, autoplay=True)

# Save the generated audio 
torchaudio.save("output.wav", torch.from_numpy(wavs[0]), 24000)


INFO:ChatTTS.core:Load from local: /Users/minkexiu/.cache/modelscope/hub/pzc163/chatTTS/
INFO:ChatTTS.core:use cpu
INFO:ChatTTS.core:vocos loaded.
INFO:ChatTTS.core:dvae loaded.
INFO:ChatTTS.core:gpt loaded.
INFO:ChatTTS.core:decoder loaded.
INFO:ChatTTS.core:tokenizer loaded.
INFO:ChatTTS.core:All initialized.
INFO:ChatTTS.core:All initialized.


UnboundLocalError: local variable 'Normalizer' referenced before assignment